In [2]:
import os
import networkx as nx
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import from_networkx

# Load the Cora dataset



In [3]:


dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Path to the folder containing subgraph GML files
folder_path = 'client_subgraphs'
sub_data_list = []
# Loop through each GML file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.gml'):
        # Read GML file using networkx
        file_path = os.path.join(folder_path, filename)
        g = nx.read_gml(file_path)

        # Convert networkx graph to PyTorch Geometric data
        sub_data = from_networkx(g)
        # Ensure node features and labels are set (this will depend on how data is stored in the GML file)

        # Example: Set dummy features and labels if not present
        if sub_data.x is None:
            num_nodes = sub_data.num_nodes
            sub_data.x = torch.randn((num_nodes, data.num_node_features))  # Replace with actual node features
        if sub_data.y is None:
            sub_data.y = torch.randint(0, dataset.num_classes, (sub_data.num_nodes,))  # Replace with actual labels
        
        sub_data_list.append(sub_data)

Processing...
Done!


In [6]:
print(sub_data_list[0])

Data(edge_index=[2, 58], num_nodes=18, x=[18, 1433], y=[18])


In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First convolutional layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        # Second convolutional layer
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Train the model
# loop through each subgraph
for i in range(0, 100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(sub_data.num_node_features, dataset.num_classes).to(device)
    sub_data_train = sub_data_list[i].to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

        # Training loop (adjust as needed)
    model.train()
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(sub_data_train)
        loss = F.nll_loss(out, sub_data.y)
        loss.backward()
        optimizer.step()


